In [3]:
import pandas as pd
from mysql import connector

# List of CSV file paths
csv_files = [
    "ap_bus_details.csv", "assam_bus_details.csv",
    "himachal_bus_details.csv", "kaac_bus_details.csv", "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", "sb_bus_details.csv", "Telangana_bus_details.csv",
    "up_bus_details.csv", "wb_bus_details.csv", "wb2_bus_details.csv"
]

# Read each CSV file into a DataFrame and combine them
dataframes = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dataframes, ignore_index=True)

# Add a unique ID column
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

# Clean the data
if 'Price' in combined_df.columns:
    combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)

if 'Seat_Availability' in combined_df.columns:
    combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Drop null values
combined_df = combined_df.dropna()

# Save the cleaned data to a CSV file
combined_csv_file = "bus_routes.csv"
combined_df.to_csv(combined_csv_file, index=False)

# Connect to MySQL
myconnection = connector.connect(
    host='127.0.0.1', user='root', passwd='123456789', database="redbus"
)

cursor = myconnection.cursor()

# Prepare SQL table schema
column_definitions = ", ".join(
    f"`{col}` {'FLOAT' if dtype == 'float64' else 'TEXT' if dtype == 'object' else 'INT'}"
    for col, dtype in zip(combined_df.columns, combined_df.dtypes)
)

# Create table
table_name = "bus_routes"
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
cursor.execute(f"CREATE TABLE {table_name} ({column_definitions});")

# Prepare parameterized query for safe insertion
columns = ', '.join(f'`{col}`' for col in combined_df.columns)
placeholders = ', '.join(['%s'] * len(combined_df.columns))
query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert data into the table
for _, row in combined_df.iterrows():
    cursor.execute(query, tuple(row))

# Commit changes and close the connection
myconnection.commit()
cursor.close()
myconnection.close()

print("Data successfully inserted into MySQL database.")


Data successfully inserted into MySQL database.
